#signs dataset

In [1]:
!wget http://www.kasprowski.pl/datasets/signs.zip

--2024-01-27 15:25:25--  http://www.kasprowski.pl/datasets/signs.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186917768 (178M) [application/zip]
Saving to: ‘signs.zip’

signs.zip           100%[===================>] 178.26M  11.7MB/s    in 16s     

2024-01-27 15:25:42 (11.0 MB/s) - ‘signs.zip’ saved [186917768/186917768]



In [2]:
!unzip "signs.zip"

Streaming output truncated to the last 5000 lines.
  inflating: signs/00030/00008_00005.ppm  
  inflating: signs/00030/00008_00006.ppm  
  inflating: signs/00030/00008_00007.ppm  
  inflating: signs/00030/00008_00008.ppm  
  inflating: signs/00030/00008_00009.ppm  
  inflating: signs/00030/00008_00010.ppm  
  inflating: signs/00030/00008_00011.ppm  
  inflating: signs/00030/00008_00012.ppm  
  inflating: signs/00030/00008_00013.ppm  
  inflating: signs/00030/00008_00014.ppm  
  inflating: signs/00030/00008_00015.ppm  
  inflating: signs/00030/00008_00016.ppm  
  inflating: signs/00030/00008_00017.ppm  
  inflating: signs/00030/00008_00018.ppm  
  inflating: signs/00030/00008_00019.ppm  
  inflating: signs/00030/00008_00020.ppm  
  inflating: signs/00030/00008_00021.ppm  
  inflating: signs/00030/00008_00022.ppm  
  inflating: signs/00030/00008_00023.ppm  
  inflating: signs/00030/00008_00024.ppm  
  inflating: signs/00030/00008_00025.ppm  
  inflating: signs/00030/00008_00026.ppm  
  i

In [94]:
import os
from skimage import io
import cv2
import numpy as np



def load_data(indir):
  sample = []
  label = []

  for class_dir in os.listdir(indir):
    # class = class_dir
    for folder in os.listdir("{}/{}".format(indir,class_dir)):
      if (folder.endswith('.ppm')):
        image = io.imread("{}/{}/{}".format(indir,class_dir,folder))
        image = cv2.resize(image, (64, 64))
      sample.append(image)
      label.append(class_dir)
  sample = np.array(sample)
  label = np.array(label)

  return sample, label


sample, label = load_data('/content/signs')
print("samples here: ", sample)
print("samples here: ", len(sample))
print("no of labels: ", len(label))

print("label here; ", set(label))



samples here:  [[[[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [255 255 255]
   [255 255 255]
   [255 255 255]]

  [[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [255 255 255]
   [255 255 255]
   [255 255 255]]

  [[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [255 255 255]
   [255 255 255]
   [255 255 255]]

  ...

  [[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [252 255 255]
   [255 255 255]
   [255 255 255]]

  [[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [249 255 255]
   [255 255 255]
   [255 255 255]]

  [[255 255 255]
   [255 255 255]
   [255 255 255]
   ...
   [245 255 255]
   [255 255 255]
   [255 255 255]]]


 [[[ 30  24  28]
   [ 30  24  28]
   [ 30  25  29]
   ...
   [ 34  29  33]
   [ 36  29  34]
   [ 37  30  34]]

  [[ 30  24  28]
   [ 30  24  29]
   [ 30  24  30]
   ...
   [ 35  29  34]
   [ 36  30  34]
   [ 37  30  35]]

  [[ 30  24  29]
   [ 29  24  29]
   [ 29  24  30]
   ...
   [ 36  30  35]
   [ 37  30  

In [95]:
sample.shape, label.shape

((26683, 64, 64, 3), (26683,))

In [104]:
import sklearn
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labe = encoder.fit_transform(label)
labels = tf.keras.utils.to_categorical(labe)
# label = np.array([0,1])





In [105]:
import sklearn
from sklearn.model_selection import train_test_split

trainSample, testSample, trainLabel, testLabel = train_test_split(sample, labels, stratify=labels, test_size=0.2)

In [69]:
trainSample.shape, labels.shape

((21346, 64, 64, 3), (26683,))

In [106]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, MaxPooling2D, Flatten, Conv2D, BatchNormalization, Dropout

model = Sequential()

model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation("relu"))

model.add(Dense(43))
model.add(Activation("softmax"))

model.summary()

optimizer = tf.keras.optimizers.legacy.RMSprop(0.0001, decay = 1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])





Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 64, 64, 16)        448       
                                                                 
 batch_normalization_21 (Ba  (None, 64, 64, 16)        64        
 tchNormalization)                                               
                                                                 
 activation_41 (Activation)  (None, 64, 64, 16)        0         
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 32, 32, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_22 (Ba  (None, 32, 32, 32)      

In [108]:
model.fit(trainSample, trainLabel, epochs=5, validation_data=(testSample, testLabel))

Epoch 1/5
668/668 [==============================] - 128s 192ms/step - loss: 1.1713 - accuracy: 0.6733 - val_loss: 0.5938 - val_accuracy: 0.8207
Epoch 2/5
668/668 [==============================] - 127s 191ms/step - loss: 0.3077 - accuracy: 0.9134 - val_loss: 0.8631 - val_accuracy: 0.7804
Epoch 3/5
668/668 [==============================] - 126s 188ms/step - loss: 0.1545 - accuracy: 0.9581 - val_loss: 0.3501 - val_accuracy: 0.9029
Epoch 4/5
668/668 [==============================] - 129s 193ms/step - loss: 0.0969 - accuracy: 0.9724 - val_loss: 0.1104 - val_accuracy: 0.9728
Epoch 5/5
668/668 [==============================] - 126s 188ms/step - loss: 0.0678 - accuracy: 0.9811 - val_loss: 0.0922 - val_accuracy: 0.9781


In [113]:
import sklearn
from sklearn.metrics import *
Results = model.predict(testSample)

print(confusion_matrix(testLabel.argmax(axis=1), Results.argmax(axis=1)))
print(classification_report(testLabel.argmax(axis=1), Results.argmax(axis=1)))
treeAcc = accuracy_score(testLabel.argmax(axis=1), Results.argmax(axis=1))
print("Cohen's Kappa: {}".format(cohen_kappa_score(testLabel.argmax(axis=1), Results.argmax(axis=1))))
print("Tree Accuracy: {:.2f}".format(treeAcc))

167/167 [==============================] - 8s 47ms/step
[[ 29   0   0 ...   0   0   0]
 [  0 274  16 ...   0   0   0]
 [  0   1 296 ...   0   0   0]
 ...
 [  0   0   0 ...  48   0   0]
 [  0   0   0 ...   0  35   0]
 [  0   0   0 ...   0   0  34]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       1.00      0.91      0.95       301
           2       0.93      0.98      0.96       301
           3       0.95      0.99      0.97       192
           4       0.98      0.98      0.98       265
           5       0.97      0.92      0.94       253
           6       1.00      1.00      1.00        60
           7       0.99      0.89      0.93       192
           8       0.82      0.99      0.90       192
           9       0.99      0.99      0.99       198
          10       1.00      1.00      1.00       271
          11       0.99      0.97      0.98       180
          12       1.00      1.00      1.00      